In [3]:
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from IPython.display import display
import os

In [4]:
file_path = "/Users/mi-yeonlim/Documents/Github/p404-customer/data/고객데이터 수정.csv"
cs = pd.read_csv(file_path, encoding="utf-8")

In [5]:
cs.columns

Index(['고객ID', '생년월일', '성별', '주소', '가입일', '고객세그먼트', '금액', '차량구분', '구매 횟수',
       '누적 구매 금액', '구매 날짜', '거래 방식', '구매 경로'],
      dtype='object')

In [6]:
cs.head()

,고객ID,생년월일,성별,주소,가입일,고객세그먼트,금액,차량구분,구매 횟수,누적 구매 금액,구매 날짜,거래 방식,구매 경로
0,14798,2002-03-04,남,인천광역시 남동구 구월동 88-10,2023-04-01,일반,35170000,준중형 세단,1,35170000,2024-04-04,신용카드,오프라인
1,kyoungsoo031,1983-01-02,남,광주광역시 서구 치평동 14-7,2025-02-09,vip,85790000,준대형 세단,5,428950000,2025-02-10,신용카드,오프라인
2,daeun026,1999-05-02,여,부산광역시 해운대구 우동 101-12,2025-02-28,신규,69500000,중형 SUV,1,69500000,2025-03-10,계좌이체,오프라인
3,minsu001,1960-07-01,남,전라남도 목포시 부주동 77-8,2023-04-28,일반,34920000,중형 SUV,1,34920000,2023-05-01,현금,오프라인
4,seunghyun020,1970-03-02,남,경기도 수원시 영통구 원천동 12-34,2023-09-18,이탈가능,50230000,준중형 세단,1,50230000,2023-10-01,신용카드,오프라인


In [7]:
import pandas as pd
from datetime import datetime

# 필요한 함수들 정의
def get_purchase_count_score(count):
    if count >= 5:
        return 5
    elif count >= 4:
        return 4
    elif count >= 3:
        return 3
    elif count >= 2:
        return 2
    else:
        return 1

def get_amount_score(amount):
    if amount >= 500000000:  # 5억 이상
        return 5
    elif amount >= 300000000:  # 3억 이상
        return 4
    elif amount >= 150000000:  # 1억 5천만 이상
        return 3
    elif amount >= 50000000:  # 5천만 이상
        return 2
    else:
        return 1  # 5천만 미만

def get_recency_score(days_since_last_purchase):
    if days_since_last_purchase <= 30:
        return 5  # 한 달 이내
    elif days_since_last_purchase <= 90:
        return 4  # 3개월 이내
    elif days_since_last_purchase <= 180:
        return 3  # 6개월 이내
    elif days_since_last_purchase <= 365:
        return 2  # 1년 이내
    else:
        return 1  # 1년 이상 미구매

def get_transaction_score(method):
    return 2 if method == "일시불" else 1

def get_purchase_path_score(path):
    return 2 if path == "온라인" else 1

# 현재 날짜 설정
current_date = datetime(2025, 3, 20)

# 예시 데이터프레임 cs
# cs = pd.read_csv('your_data.csv')  # 실제 데이터프레임을 여기에 로드

# 1. 구매 횟수 점수
cs['구매 횟수 점수'] = cs['구매 횟수'].apply(get_purchase_count_score)

# 2. 누적 구매 금액 점수
cs['누적 구매 금액 점수'] = cs['누적 구매 금액'].apply(get_amount_score)

# 3. 구매 최근성 점수
cs['구매 날짜'] = pd.to_datetime(cs['구매 날짜'])
cs['마지막 구매 이후 일수'] = (current_date - cs['구매 날짜']).dt.days
cs['구매 최근성 점수'] = cs['마지막 구매 이후 일수'].apply(get_recency_score)

# 4. 거래 방식 점수
cs['거래 방식 점수'] = cs['거래 방식'].apply(get_transaction_score)

# 5. 구매 경로 점수
cs['구매 경로 점수'] = cs['구매 경로'].apply(get_purchase_path_score)

# 총점 계산
cs['총점'] = (cs['구매 횟수 점수'] + 
             cs['누적 구매 금액 점수'] + 
             cs['구매 최근성 점수'] + 
             cs['거래 방식 점수'] + 
             cs['구매 경로 점수'])

# 고객 세그먼트 재생성
def classify_customer(score, recency_days, purchase_count):
    # 신규 조건: 최근 5개월 이내 구매 && 구매 횟수 1회인 고객
    if recency_days <= 180 and purchase_count == 1:  # 5개월 이내 && 구매 횟수 1
        return "신규"
    
    # 기존 조건: 총점 기준으로 세그먼트 분류
    if score >= 13:
        return "VIP"  # 13점 이상은 VIP
    elif score >= 8:
        return "일반"  # 8점 이상 13점 미만은 일반
    else:
        return "이탈가능"  # 7점 이하인 경우 이탈가능

# 고객 세그먼트 적용
cs['등급'] = cs.apply(lambda row: classify_customer(row['총점'], row['마지막 구매 이후 일수'], row['구매 횟수']), axis=1)

# 결과 확인
print(cs[['고객ID', '구매 횟수 점수', '누적 구매 금액 점수', '구매 최근성 점수', 
          '거래 방식 점수', '구매 경로 점수', '총점', '등급']].head())


           고객ID  구매 횟수 점수  누적 구매 금액 점수  구매 최근성 점수  거래 방식 점수  구매 경로 점수  총점  \
0         14798         1            1          2         1         1   6   
1  kyoungsoo031         5            4          4         1         1  15   
2      daeun026         1            2          5         1         1  10   
3      minsu001         1            1          1         1         1   5   
4  seunghyun020         1            2          1         1         1   6   

     등급  
0  이탈가능  
1   VIP  
2    신규  
3  이탈가능  
4  이탈가능  


In [8]:
print(cs["총점"].value_counts())

총점
7     45
8     35
10    34
9     34
11    31
13    25
6     24
12    23
14    17
5     12
15     9
16     3
Name: count, dtype: int64


In [9]:
cs.head()

,고객ID,생년월일,성별,주소,가입일,고객세그먼트,금액,차량구분,구매 횟수,누적 구매 금액,...,거래 방식,구매 경로,구매 횟수 점수,누적 구매 금액 점수,마지막 구매 이후 일수,구매 최근성 점수,거래 방식 점수,구매 경로 점수,총점,등급
0,14798,2002-03-04,남,인천광역시 남동구 구월동 88-10,2023-04-01,일반,35170000,준중형 세단,1,35170000,...,신용카드,오프라인,1,1,350,2,1,1,6,이탈가능
1,kyoungsoo031,1983-01-02,남,광주광역시 서구 치평동 14-7,2025-02-09,vip,85790000,준대형 세단,5,428950000,...,신용카드,오프라인,5,4,38,4,1,1,15,VIP
2,daeun026,1999-05-02,여,부산광역시 해운대구 우동 101-12,2025-02-28,신규,69500000,중형 SUV,1,69500000,...,계좌이체,오프라인,1,2,10,5,1,1,10,신규
3,minsu001,1960-07-01,남,전라남도 목포시 부주동 77-8,2023-04-28,일반,34920000,중형 SUV,1,34920000,...,현금,오프라인,1,1,689,1,1,1,5,이탈가능
4,seunghyun020,1970-03-02,남,경기도 수원시 영통구 원천동 12-34,2023-09-18,이탈가능,50230000,준중형 세단,1,50230000,...,신용카드,오프라인,1,2,536,1,1,1,6,이탈가능


In [10]:
print(cs["등급"].value_counts())

등급
일반      143
이탈가능     79
VIP      54
신규       16
Name: count, dtype: int64


In [11]:
cs.to_csv("/Users/mi-yeonlim/Documents/Github/p404-customer/data/updated_customer.csv", encoding="utf-8", index=False)

In [80]:
# 🔹 원하는 데이터 개수
target_size = 50000  # 목표 개수
current_size = len(cs)  # 현재 개수

# 🔹 증강할 배수 계산
repeat_factor = target_size // current_size  # 반복 배수
extra_samples = target_size % current_size  # 추가 샘플 개수

# 🔹 데이터 반복하여 증가
cs_augmented = pd.concat([cs] * repeat_factor, ignore_index=True)

# 🔹 추가 샘플 랜덤 선택하여 합치기
cs_extra = cs.sample(n=extra_samples, replace=True, random_state=42)
cs_augmented = pd.concat([cs_augmented, cs_extra], ignore_index=True)

In [81]:

import random

# 🔹 현재 날짜 기준 (분석 기준일: 2025-03-20)
current_date = pd.Timestamp("2025-03-20")

# 🔹 차량구분별 가격 범위 (시세 기반, 단위: 원)
vehicle_price_ranges = {
    "준중형 세단": (20000000, 40000000),
    "준대형 세단": (35000000, 60000000),
    "중형 SUV": (35000000, 70000000),
    "중형 세단": (30000000, 55000000),
    "대형 세단": (50000000, 90000000),
    "준중형 SUV": (25000000, 45000000),
    "대형 SUV": (60000000, 120000000),
    "준중형 해치백": (15000000, 30000000)
}

# 🔹 고객 세그먼트 설정 (랜덤 분류)
customer_segments = ["일반", "VIP", "이탈 가능", "신규"]

# 🔹 데이터 변형
for i in range(len(cs)):
    # 🔹 차량구분 랜덤 변경
    vehicle_type = random.choice(list(vehicle_price_ranges.keys()))
    cs.loc[i, "차량구분"] = vehicle_type

    # 🔹 금액 변형 (해당 차량구분 시세 ±10% 내에서 랜덤 조정)
    min_price, max_price = vehicle_price_ranges[vehicle_type]
    base_price = random.randint(min_price, max_price)
    adjusted_price = base_price * np.random.uniform(0.9, 1.1)
    cs.loc[i, "금액"] = round(adjusted_price, -4)  # 1만원 단위 반올림

    # 🔹 구매 횟수 변형 (1~5회 랜덤 설정)
    cs.loc[i, "구매 횟수"] = max(1, np.random.randint(1, 6))  # 최소 1회 보장

    # 🔹 가입일 및 구매 날짜 변형 (최대 2년 전 ~ 현재까지)
    random_days_offset = np.random.randint(0, 730)
    cs.loc[i, "구매 날짜"] = current_date - pd.to_timedelta(random_days_offset, unit="D")
    cs.loc[i, "가입일"] = cs.loc[i, "구매 날짜"] - pd.to_timedelta(np.random.randint(30, 365), unit="D")

    # 🔹 고객 세그먼트 랜덤 할당
    cs.loc[i, "고객세그먼트"] = random.choice(customer_segments)

# 🔹 누적 구매 금액 = 금액 × 구매 횟수
cs["누적 구매 금액"] = cs["금액"] * cs["구매 횟수"]

# 🔹 결과 확인
print("\n✅ [데이터 증강 후 샘플]")
print(cs.head())

# 🔹 파일 저장
output_path = "/Users/mi-yeonlim/Documents/Github/p404-customer/data/cs_augmented_final.csv"
cs.to_csv(output_path, index=False, encoding="utf-8")
print(f"\n📁 최종 변형된 데이터 저장 완료: {output_path}")



✅ [데이터 증강 후 샘플]
           고객ID        생년월일 성별                     주소                  가입일  \
0         14798  2002-03-04  남    인천광역시 남동구 구월동 88-10  2023-03-27 00:00:00   
1  kyoungsoo031  1983-01-02  남      광주광역시 서구 치평동 14-7  2023-01-18 00:00:00   
2      daeun026  1999-05-02  여   부산광역시 해운대구 우동 101-12  2024-06-08 00:00:00   
3      minsu001  1960-07-01  남      전라남도 목포시 부주동 77-8  2023-11-11 00:00:00   
4  seunghyun020  1970-03-02  남  경기도 수원시 영통구 원천동 12-34  2023-12-10 00:00:00   

  고객세그먼트        금액     차량구분  구매 횟수   누적 구매 금액                구매 날짜 거래 방식 구매 경로  
0     신규  16790000  준중형 해치백      1   16790000  2024-02-27 00:00:00  신용카드  오프라인  
1  이탈 가능  26440000  준중형 해치백      3   79320000  2023-03-25 00:00:00  신용카드  오프라인  
2    VIP  27140000  준중형 해치백      4  108560000  2024-07-18 00:00:00  계좌이체  오프라인  
3     신규  34680000  준중형 SUV      1   34680000  2024-02-15 00:00:00    현금  오프라인  
4  이탈 가능  36540000  준중형 SUV      2   73080000  2024-11-03 00:00:00  신용카드  오프라인  

📁 최종 변형된 데이터 저장 완료: /Users/

In [60]:
cs["가입일"] = pd.to_datetime(cs["가입일"], errors="coerce")
cs["구매 날짜"] = pd.to_datetime(cs["구매 날짜"], errors="coerce")

In [61]:
cs["가입일"] = cs["가입일"].dt.strftime("%Y-%m-%d")
cs["구매 날짜"] = cs["구매 날짜"].dt.strftime("%Y-%m-%d")

In [62]:
cs

,고객ID,생년월일,성별,주소,가입일,고객세그먼트,금액,차량구분,구매 횟수,누적 구매 금액,구매 날짜,거래 방식,구매 경로
0,14798,2002-03-04,남,인천광역시 남동구 구월동 88-10,2023-03-24,VIP,54310000,준대형 세단,4,217240000,2024-01-08,신용카드,오프라인
1,kyoungsoo031,1983-01-02,남,광주광역시 서구 치평동 14-7,2024-02-23,VIP,51720000,중형 세단,3,155160000,2024-09-01,신용카드,오프라인
2,daeun026,1999-05-02,여,부산광역시 해운대구 우동 101-12,2023-03-16,이탈 가능,16550000,준중형 해치백,2,33100000,2023-11-27,계좌이체,오프라인
3,minsu001,1960-07-01,남,전라남도 목포시 부주동 77-8,2023-07-30,VIP,28110000,준중형 세단,5,140550000,2024-07-14,현금,오프라인
4,seunghyun020,1970-03-02,남,경기도 수원시 영통구 원천동 12-34,2024-02-02,이탈 가능,65010000,대형 SUV,1,65010000,2025-01-16,신용카드,오프라인
...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,jihungim,1958-07-11,여,전라북도 전주시 완산구 덕진동 33-7,2024-03-26,일반,25930000,준중형 해치백,3,77790000,2024-10-05,신용카드,온라인
288,jinho50,1976-06-27,여,충청남도 천안시 동남구 성성동 22-3,2023-01-27,VIP,24070000,준중형 세단,4,96280000,2023-05-03,현금,오프라인
289,jihun41,1968-01-22,여,경상남도 창원시 의창구 봉곡동 48-6,2024-05-10,일반,48720000,준대형 세단,5,243600000,2024-07-12,계좌이체,온라인
290,yeongil47,1984-01-10,남,대구광역시 수성구 범어동 55-3,2022-11-17,VIP,41510000,준중형 세단,5,207550000,2023-05-12,현금,온라인


In [74]:
file_path = "/Users/mi-yeonlim/Documents/Github/p404-customer/data/cs_augmented_final.csv"
df = pd.read_csv(file_path, encoding="utf-8")

In [75]:
cs.columns

Index(['고객ID', '생년월일', '성별', '주소', '가입일', '고객세그먼트', '금액', '차량구분', '구매 횟수',
       '누적 구매 금액', '구매 날짜', '거래 방식', '구매 경로'],
      dtype='object')

In [76]:
cs

,고객ID,생년월일,성별,주소,가입일,고객세그먼트,금액,차량구분,구매 횟수,누적 구매 금액,구매 날짜,거래 방식,구매 경로
0,14798,2002-03-04,남,인천광역시 남동구 구월동 88-10,2023-03-24,VIP,54310000,준대형 세단,4,217240000,2024-01-08,신용카드,오프라인
1,kyoungsoo031,1983-01-02,남,광주광역시 서구 치평동 14-7,2024-02-23,VIP,51720000,중형 세단,3,155160000,2024-09-01,신용카드,오프라인
2,daeun026,1999-05-02,여,부산광역시 해운대구 우동 101-12,2023-03-16,이탈 가능,16550000,준중형 해치백,2,33100000,2023-11-27,계좌이체,오프라인
3,minsu001,1960-07-01,남,전라남도 목포시 부주동 77-8,2023-07-30,VIP,28110000,준중형 세단,5,140550000,2024-07-14,현금,오프라인
4,seunghyun020,1970-03-02,남,경기도 수원시 영통구 원천동 12-34,2024-02-02,이탈 가능,65010000,대형 SUV,1,65010000,2025-01-16,신용카드,오프라인
...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,jihungim,1958-07-11,여,전라북도 전주시 완산구 덕진동 33-7,2024-03-26,일반,25930000,준중형 해치백,3,77790000,2024-10-05,신용카드,온라인
288,jinho50,1976-06-27,여,충청남도 천안시 동남구 성성동 22-3,2023-01-27,VIP,24070000,준중형 세단,4,96280000,2023-05-03,현금,오프라인
289,jihun41,1968-01-22,여,경상남도 창원시 의창구 봉곡동 48-6,2024-05-10,일반,48720000,준대형 세단,5,243600000,2024-07-12,계좌이체,온라인
290,yeongil47,1984-01-10,남,대구광역시 수성구 범어동 55-3,2022-11-17,VIP,41510000,준중형 세단,5,207550000,2023-05-12,현금,온라인


## 경제심리지수란?

경제심리지수는 소비자들이 현재와 미래의 경제 상황에 대해 어떻게 느끼고 있는지를 나타내는 지표입니다.

## 용어 구분

### 1. 경제심리지수 (원계열)
- 측정된 경제심리지수의 **원래 값**을 의미합니다.
- 이 값에는 **장기 추세** (예: 경제 성장 추세)와 **단기 변동** (예: 경기 순환 등)이 모두 포함되어 있습니다.

### 2. 경제심리지수 (순환변동치)
- 원계열에서 **장기 추세를 제거**하고, **순환(주기적인 변동)**만을 추출한 값입니다.
- 이를 통해 경제의 **단기 경기 순환**이나 변동 패턴을 더 명확하게 파악할 수 있습니다.

## 구분하는 이유

- **원계열**은 경제 전반의 장기적 흐름과 단기 변동이 혼합되어 있어, 단기 경기 변동을 정확하게 파악하기 어렵습니다.
- **순환변동치**는 단기 변동에 집중하여 경기 사이클(호황과 불황)을 보다 명확하게 분석할 수 있게 해줍니다.


In [7]:
# 데이터프레임 이름이 df라고 가정
cs = cs.rename(columns={
    '원자료': '경제심리지수 (원계열)',
    '원자료.1': '경제심리지수 (순환변동치)'
})


In [8]:
cs.columns

Index(['변환', '경제심리지수 (원계열)', '경제심리지수 (순환변동치)'], dtype='object')

In [9]:
cs.to_csv("/Users/mi-yeonlim/Documents/Github/p404-customer/data/경제심리지수.csv", encoding="utf-8", index=False)

In [10]:
cs.head()

,변환,경제심리지수 (원계열),경제심리지수 (순환변동치)
0,2023/01,90.7,93.4
1,2023/02,91.8,93.0
2,2023/03,91.9,92.8
3,2023/04,94.5,92.9
4,2023/05,94.9,93.1


In [28]:
file_path = "/Users/mi-yeonlim/Documents/Github/p404-customer/data/고객데이터 수정.csv"
df_1 = pd.read_csv(file_path, encoding="utf-8")

In [29]:
df_1.columns

Index(['생년월일', '성별', '등급', '차 종', '구매한 제품', '구매 날짜', '금액', '거래 방식', '구매 횟수',
       '구매 경로', '주소'],
      dtype='object')

In [22]:
file_path = "/Users/mi-yeonlim/Documents/Github/p404-customer/data/지급결제동향_14103901.csv"
pv = pd.read_csv(file_path, encoding="utf-8")

In [23]:
pv.head()

,변환,원자료,원자료.1,원자료.2,원자료.3,원자료.4,원자료.5,원자료.6,원자료.7,원자료.8,...,원자료.72,원자료.73,원자료.74,원자료.75,원자료.76,원자료.77,원자료.78,원자료.79,원자료.80,원자료.81
0,2023/01,"84,391.0","4,044,620,672.1","20,559.0","862,492,411.1","10,546.0","271,789,394.5",11.0,"193,359.3","6,448.0",...,0.0,0.0,"18,855.0","219,363,579.3",762.0,"81,536,556.2","1,059.0","264,490,143.7","105,664.0","3,298,239,938.0"
1,2023/02,"85,826.0","3,916,028,594.0","21,850.0","926,309,911.7","11,378.0","314,021,316.1",11.0,"285,866.0","7,567.0",...,0.0,0.0,"22,245.0","185,876,184.3",757.0,"78,348,070.0","1,218.0","261,438,313.1","106,592.0","3,477,392,600.2"
2,2023/03,"97,592.0","4,545,478,079.0","24,710.0","1,078,234,224.4","13,284.0","354,361,750.2",4.0,"40,704.7","7,258.0",...,0.0,0.0,"29,804.0","163,946,724.9",773.0,"78,031,059.0","1,354.0","404,392,726.8","120,820.0","3,731,932,275.8"
3,2023/04,"90,848.0","4,116,148,621.1","23,997.0","1,016,948,005.8","12,275.0","332,719,774.0",8.0,"86,499.6","7,335.0",...,0.0,0.0,"23,167.0","193,275,345.6",769.0,"77,094,123.0","1,029.0","283,348,298.2","111,386.0","3,487,670,264.5"
4,2023/05,"89,537.0","3,973,341,618.7","22,718.0","971,390,200.1","11,795.0","344,628,442.1",8.0,"22,801.1","7,883.0",...,0.0,0.0,"22,999.0","197,822,281.9",747.0,"74,180,422.0","1,172.0","144,049,260.0","113,621.0","3,537,443,259.2"


주 : 
- 2019년 이전 지수는 소수점 이하 3자리, 2020년 이후 지수는 소수점 이하 2자리로 작성 
- 2020년기준 가중치로서 이전자료(~2019년)는 동일가중치를 적용 할 수 없음
- 계절적 영향으로 일정기간 출회되지 않는 계절 농산물 등 11개 품목의 지수작성 방법을 ‘17년 1월분부터 변경함
    * (당초) 이월 : 미출회기간 중의 지수를 출회기간의 마지막 달의 지수와 동일하게 처리하는 방식
  ** (변경) 대체 : 품목군별로 계절품목을 제외한 실제 거래되는 품목 전체의 지수 변동률을 계절 품목에 적용
 - 11개 계절품목은 아래와 같음(괄호 내는 미출회기간)
  · 과실 : 복숭아(10월~익년6월), 포도(11월~익년7월), 감, 귤(4월~9월), 오렌지(7월~12월), 
             참외(9월~익년 2월), 수박(9월~익년4월), 딸기(6월~11월), 체리(3월~5월, 9월~11월) 
  · 채소 : 열무(11월~익년 2월)
  · 수산물 : 굴(4월~9월)

(통계 이용시 유의사항)
- 한국은행은 직접 작성한 통계가 아니더라도 이용자 편의차원에서 일부 제공하고 있습니다.
- 5일 내외의 수록 시차가 있으므로 필요시 통계 생산 기관을 이용하시기 바랍니다. 

In [24]:
file_path = "/Users/mi-yeonlim/Documents/Github/p404-customer/data/소비자물가지수_14103710.csv"
cpi = pd.read_csv(file_path, encoding="utf-8")

In [25]:
cpi.head()

,변환,원자료,원자료.1,원자료.2,원자료.3,원자료.4,원자료.5,원자료.6,원자료.7,원자료.8,...,원자료.571,원자료.572,원자료.573,원자료.574,원자료.575,원자료.576,원자료.577,원자료.578,원자료.579,원자료.580
0,2023/01,110.07,115.37,115.46,114.10,94.27,98.32,92.62,111.66,99.32,...,99.07,109.59,135.36,99.07,100,92.28,100.0,100.92,104.87,105.42
1,2023/02,110.33,115.91,115.82,116.27,94.60,97.68,93.53,112.04,97.80,...,99.07,109.59,135.66,99.07,100,92.28,100.0,100.92,104.87,105.72
2,2023/03,110.52,116.46,116.30,116.19,93.99,96.17,93.34,113.04,98.08,...,96.63,109.59,135.66,97.06,100,92.28,100.0,100.92,104.87,106.43
3,2023/04,110.77,115.90,115.68,116.69,93.43,96.16,93.32,113.58,98.42,...,96.64,109.59,142.10,97.06,100,92.28,100.0,100.92,104.87,106.74
4,2023/05,111.13,115.99,115.84,116.72,94.04,96.02,93.00,113.70,98.19,...,96.67,109.59,142.10,97.06,100,92.28,100.0,100.92,104.87,106.83
